In [ ]:
from PIL import Image
import torch
from torchvision.transforms import v2
import os
import glob
import posixpath
import torchvision.transforms.functional as TF


In [ ]:
# flip all images horizontally and save to output folder
cwd = os.getcwd()

source_filters = ['./datasample/s/cc/*.jpg', './datasample/m/cc/*.jpg', './datasample/b/cc/*.jpg']
destination_folders = ['./datasample/s/cc/aug', './datasample/m/cc/aug', './datasample/b/cc/aug']

for i in range(len(source_filters)):
    source_filter = source_filters[i]
    source_label_filter = source_filter.replace(".jpg", "_label.jpg")
    destination_folder = destination_folders[i]
    print("\n", len(glob.glob(source_filter)), source_filter, source_label_filter, destination_folder)
    if not os.path.isdir(os.path.join(cwd,destination_folder)):
        os.mkdir(os.path.join(cwd,destination_folder))
    
    all_images = sorted(glob.glob(source_filter),
                        key = lambda i: (len(posixpath.basename(i)), i))
    all_labels = sorted(glob.glob(source_label_filter),
                        key = lambda i: (len(posixpath.basename(i)), i))
    all_inputs = []
    
    for i in all_images:
        if i in all_labels:
            continue 
        else:
            all_inputs.append(i)
    
    for input, label in zip(all_inputs, all_labels):
        print('.', end='')
        im1 = Image.open(input)
        im2 = Image.open(label)
        
        i, j, h, w = v2.RandomCrop.get_params(im1, output_size=(768, 512))
        image = TF.crop(im1, i, j, h, w)
        mask = TF.crop(im2, i, j, h, w)
        resize = v2.Resize(size=(1024, 1024))
        image1 = resize(image)
        mask1 = resize(mask)
        
        base1, ext1 = os.path.splitext(os.path.basename(input))  
        ext1 = ext1.lower()  
        base2, ext2 = os.path.splitext(os.path.basename(label))  
        ext2 = ext2.lower()  
    
        new_path1 = os.path.join(destination_folder, f"{base1}rc{ext1}")
        image1.save(new_path1)
    
        new_path2 = os.path.join(destination_folder, f"{base2[:-6]}rc_label{ext2}") 
        mask1.save(new_path2)
        
print(".")
print("finished")